<a href="https://colab.research.google.com/github/rahiakela/transformers-research-and-practice/blob/main/natural-language-processing-with-transformers/06-summarization/summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Summarization

Text summarization is a
difficult task for neural language models, including transformers. Despite these challenges,
text summarization offers the prospect for domain experts to significantly
speed up their workflows and is used by enterprises to condense internal knowledge,
summarize contracts, automatically generate content for social media releases,
and more.

Summarization is a classic
sequence-to-sequence (seq2seq) task with an input text and a target text.

##Setup

In [ ]:
!pip -q install transformers[sentencepiece]
!pip -q install datasets
!pip -q install sacrebleu
!pip -q install rouge_score
!pip -q install py7zr

In [2]:
from transformers import pipeline, set_seed
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer

from datasets import load_dataset, load_metric

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

##CNN/DailyMail Dataset

The CNN/DailyMail dataset consists of around 300,000 pairs of news articles and
their corresponding summaries, composed from the bullet points that CNN and the
DailyMail attach to their articles.

An important aspect of the dataset is that the summaries are abstractive and not extractive, which means that they consist of new
sentences instead of simple excerpts.

We’ll use version 3.0.0, which is a nonanonymized version set up for summarization.

In [ ]:
dataset = load_dataset("cnn_dailymail", version="3.0.0")

In [ ]:
print(f"Features: {dataset['train'].column_names}")

Features: ['article', 'highlights', 'id']


Let’s look at an excerpt from an article:

In [ ]:
sample = dataset["train"][1]

print(f"""Article (excerpt of 500 characters, total length: {len(sample['article'])})""")
print(sample["article"])

print(f"\nSummary (length: {len(sample['highlights'])})")
print(sample["highlights"])

Article (excerpt of 500 characters, total length: 3192)
(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay. The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds. The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover. The 26-year-old Bolt has now collected eight gold medals at world championships, equaling the record held by American trio Carl Lewis, Michael Johnson and Allyson Felix, not to mention the small matter of six Olympic titles. The relay triumph followed individual successes in the 100 and 200 meters in the Russian capital. "I'm proud of myself and I'll continue to work to dominate for as long as possible," Bolt said, having previously expressed his intention to carry 

We see that the articles can be very long compared to the target summary; in this particular
case the difference is 17-fold.

Long articles pose a challenge to most transformer
models since the context size is usually limited to 1,000 tokens or so, which is
equivalent to a few paragraphs of text. The standard, yet crude way to deal with this
for summarization is to simply truncate the texts beyond the model’s context size.

##Text Summarization Pipelines

Let’s see how a few of the most popular transformer models for summarization perform
by first looking qualitatively at the outputs for the preceding example.

In [ ]:
sample_text = dataset["train"][1]["article"][:2000]  # restrict the input text to 2,000 characters

# We'll collect the generated summaries of each model in a dictionary
summaries = {}

Let's differentiate the end of a sentence
from punctuation that occurs in abbreviations

In [ ]:
string = "The U.S. are a country. The U.N. is an organization."
sent_tokenize(string)

['The U.S. are a country.', 'The U.N. is an organization.']

###Summarization Baseline

A common baseline for summarizing news articles is to simply take the first three
sentences of the article.

In [ ]:
def three_sentence_summary(text):
  return "\n".join(sent_tokenize(text)[:3])

In [ ]:
summaries["baseline"] = three_sentence_summary(sample_text)

###GPT-2

One of GPT-2’s surprising features is that we can also use it to generate summaries
by simply appending `TL;DR` at the end of the input text. 

The expression `TL;DR` (too long; didn’t read) is often used on platforms like Reddit to indicate a short version
of a long post.

In [ ]:
set_seed(42)

pipe = pipeline("text-generation", model="gpt2")

gpt2_query = sample_text + "\nTL;DR:\n"
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)
summaries["gpt2"] = "\n".join(sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query):]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


###T5

Next let’s try the T5 transformer.

The T5 checkpoints are trained on a mixture of unsupervised data (to
reconstruct masked words) and supervised data for several tasks, including summarization.
These checkpoints can thus be directly used to perform summarization
without fine-tuning by using the same prompts used during pretraining.

In this
framework, the input format for the model to summarize a document is "summarize:`<ARTICLE>`", and for translation it looks like "translate English to German:`<TEXT>`".

In [ ]:
pipe = pipeline("summarization", model="t5-large")

pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

###BART

BART also uses an encoder-decoder architecture and is trained to reconstruct corrupted inputs. It combines the pretraining schemes of BERT and GPT-2.

In [ ]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")

pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

###PEGASUS

Like BART, PEGASUS is an encoder-decoder transformer.its pretraining objective is to predict masked sentences in multisentence texts.

The
authors argue that the closer the pretraining objective is to the downstream task, the
more effective it is. 

With the aim of finding a pretraining objective that is closer to
summarization than general language modeling, they automatically identified, in a
very large corpus, sentences containing most of the content of their surrounding
paragraphs (using summarization evaluation metrics as a heuristic for content
overlap) and pretrained the PEGASUS model to reconstruct these sentences, thereby
obtaining a state-of-the-art model for text summarization.



In [ ]:
pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail")

pipe_out = pipe(sample_text)
summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>", ".\n")

### Comparing Different Summaries

Now that we have generated summaries with four different models, let’s compare the results.

Keep in mind that one model has not been trained on the dataset at all
(GPT-2), one model has been fine-tuned on this task among others (T5), and two
models have exclusively been fine-tuned on this task (BART and PEGASUS).

Let’s
have a look at the summaries these models have generated:

In [ ]:
print("GROUND TRUTH")
print(dataset["train"][1]["highlights"])
print("")

for model_name in summaries:
  print(model_name.upper())
  print("-----------------------------------")
  print(summaries[model_name])
  print("")

GROUND TRUTH
Usain Bolt wins third gold of world championship .
Anchors Jamaica to 4x100m relay victory .
Eighth gold at the championships for Bolt .
Jamaica double up in women's 4x100m relay .

BASELINE
-----------------------------------
(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay.
The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds.
The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover.

GPT2
-----------------------------------
Usain Bolt triumphed after a difficult final, but then he got into trouble when Canada made him try to block.
It's the last match he's held all of last year with the men, who were later picked in the third and fourth rounds of the men's 100m an

The first thing we notice by looking at the model outputs is that the summary generated
by GPT-2 is quite different from the others. Instead of giving a summary of the
text, it summarizes the characters.

Comparing the other three model summaries against the ground truth, we see that
there is remarkable overlap, with PEGASUS’s output bearing the most striking
resemblance.

Now that we have inspected a few models, let’s try to decide which one we would use
in a production setting.

However, this
is not a systematic way of determining the best model! Ideally, we would define a metric, measure it for all models on some benchmark dataset, and choose the one
with the best performance.

But how do you define a metric for text generation?

The
standard metrics that we’ve seen, like accuracy, recall, and precision, are not easy to
apply to this task.

##Measuring the Quality of Generated Text

Good evaluation metrics are important, since we use them to measure the performance
of models not only when we train them but also later, in production.

Measuring performance on a text generation task is not as easy as with standard classification
tasks such as sentiment analysis or named entity recognition.

Two of the most common metrics used to evaluate generated text are **BLEU** and **ROUGE**.

###BLEU

The idea of BLEU is simple: instead of looking at how many of the tokens in the generated texts are perfectly aligned with the reference text tokens, we look at words or n-grams. 

BLEU is a precision-based metric, which means that when we compare the
two texts we count the number of words in the generation that occur in the reference and divide it by the length of the reference.

The weakness of the BLEU metric is that it expects the text to already
be tokenized. This can lead to varying results if the exact same method for text tokenization
is not used. 

The SacreBLEU metric addresses this issue by internalizing the
tokenization step; for this reason, it is the preferred metric for benchmarking.

In [ ]:
bleu_metric = load_metric("sacrebleu")

The `bleu_metric` object is an instance of the Metric class, and works like an aggregator: you can add single instances with `add()` or whole batches via `add_batch()`.

In [ ]:
bleu_metric.add(prediction="the the the the the the", reference=["the cat is on the mat"])

results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
results["precisions"] = [np.round(p, 2) for p in results["precisions"]]

pd.DataFrame.from_dict(results, orient="index", columns=["Value"])

,Value
score,0.0
counts,"[2, 0, 0, 0]"
totals,"[6, 5, 4, 3]"
precisions,"[33.33, 0.0, 0.0, 0.0]"
bp,1.0
sys_len,6
ref_len,6


We can see the precision of the 1-gram is indeed 2/6, whereas the precisions for the 2/3/4-grams are all 0.

This means the geometric mean is zero, and
thus also the BLEU score. 

Let’s look at another example where the prediction is
almost correct:

In [ ]:
bleu_metric.add(prediction="the cat is on mat", reference=["the cat is on the mat"])

results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
results["precisions"] = [np.round(p, 2) for p in results["precisions"]]

pd.DataFrame.from_dict(results, orient="index", columns=["Value"])

,Value
score,57.893007
counts,"[5, 3, 2, 1]"
totals,"[5, 4, 3, 2]"
precisions,"[100.0, 75.0, 66.67, 50.0]"
bp,0.818731
sys_len,5
ref_len,6


We observe that the precision scores are much better. The 1-grams in the prediction
all match, and only in the precision scores do we see that something is off.

The BLEU score is widely used for evaluating text, especially in machine translation,
since precise translations are usually favored over translations that include all possible
and appropriate words.

There are other applications, such as summarization, where the situation is different.
There, we want all the important information in the generated text, so we favor high
recall. 

This is where the ROUGE score is usually used.

###ROUGE

The ROUGE score was specifically developed for applications like summarization
where high recall is more important than just precision.

The approach is very similar
to the BLEU score in that we look at different n-grams and compare their occurrences
in the generated text and the reference texts. 

The difference is that with ROUGE we
check how many n-grams in the reference text also occur in the generated text.

In the Datasets implementation, two variations of ROUGE are calculated: 

* one calculates the score per sentence and averages it for the summaries (ROUGE-L)

* the other calculates it directly over the whole summary (ROUGE-Lsum).

In [ ]:
rouge_metric = load_metric("rouge")

Let’s apply the ROUGE score to all the summaries generated by the models:

In [ ]:
reference = dataset["train"][1]["highlights"]
records = []
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

for model_name in summaries:
  rouge_metric.add(prediction=summaries[model_name], reference=reference)
  score = rouge_metric.compute()
  rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
  records.append(rouge_dict)

pd.DataFrame.from_records(records, index=summaries.keys())

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.303571,0.090909,0.214286,0.232143
gpt2,0.212766,0.021739,0.127660,0.148936
t5,0.486486,0.222222,0.378378,0.486486
bart,0.582278,0.207792,0.455696,0.506329
pegasus,0.866667,0.655172,0.800000,0.833333


These results are obviously not very reliable as we only looked at a single sample, but
we can compare the quality of the summary for that one example.



##Evaluating PEGASUS

We now have all the pieces in place to evaluate the model properly: we have a dataset
with a test set from CNN/DailyMail, we have a metric with ROUGE, and we have a
summarization model. We just need to put the pieces together.

Let’s first evaluate the performance of the three-sentence baseline:

In [ ]:
def evaluate_summaries_baseline(dataset, metric, column_text="article", column_summary="highlights"):
  summaries = [three_sentence_summary(text) for text in dataset[column_text]]
  metric.add_batch(predictions=summaries, references=dataset[column_summary])
  score = metric.compute()
  return score

Since the test fraction of the
CNN/DailyMail dataset consists of roughly 10,000 samples, generating summaries for
all these articles takes a lot of time.

For the purpose of keeping the calculations relatively
fast, we’ll subsample the test set and run the evaluation on 1,000 samples
instead. This should give us a much more stable score estimation while completing in
less than one hour on a single GPU for the PEGASUS model:

In [ ]:
test_sampled = dataset["test"].shuffle(seed=42).select(range(1000))

score = evaluate_summaries_baseline(test_sampled, rouge_metric)
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

pd.DataFrame.from_dict(rouge_dict, orient="index", columns=["baseline"]).T

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/cnn_dailymail/default/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234/cache-6ffd8e582e03c755.arrow


,rouge1,rouge2,rougeL,rougeLsum
baseline,0.388071,0.170554,0.247146,0.354972


The scores are mostly worse than on the previous example, but still better than those
achieved by GPT-2! 

Now let’s implement the same evaluation function for evaluating
the PEGASUS model:

In [ ]:
def chunks(list_of_elements, batch_size):
  """Yield successive batch-sized chunks from list_of_elements."""
  for i in range(0, len(list_of_elements), batch_size):
      yield list_of_elements[i : i + batch_size]

In [ ]:
def evaluate_summaries_pegasus(dataset, metric, model, tokenizer, batch_size=16, device=device, column_text="article", column_summary="highlights"):
  article_batches = list(chunks(dataset[column_text], batch_size))
  target_batches = list(chunks(dataset[column_summary], batch_size))

  for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
      
      inputs = tokenizer(article_batch, max_length=1024,  truncation=True, padding="max_length", return_tensors="pt")
      
      summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                  attention_mask=inputs["attention_mask"].to(device), 
                                  length_penalty=0.8, num_beams=8, max_length=128)
      
      decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]
      decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
      metric.add_batch(predictions=decoded_summaries, references=target_batch)
      
  score = metric.compute()
  return score

Let’s now load the model again with the `AutoModelForSeq2SeqLM` class, used for `seq2seq` generation tasks, and evaluate it:

In [ ]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
score = evaluate_summaries_pegasus(test_sampled, rouge_metric, model, tokenizer, batch_size=8)
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

pd.DataFrame.from_dict(rouge_dict, index=["pegasus"])

These numbers are very close to the published results. One thing to note here is that
the loss and per-token accuracy are decoupled to some degree from the ROUGE
scores. The loss is independent of the decoding strategy, whereas the ROUGE score is
strongly coupled.

Since ROUGE and BLEU correlate better with human judgment than loss or accuracy,
we should focus on them and carefully explore and choose the decoding strategy
when building text generation models. These metrics are far from perfect, however,
and one should always consider human judgments as well.

##Training Summarization Model

Now, let’s
put this to use to train a custom text summarization model! For our application, we’ll
use the [SAMSum dataset](https://huggingface.co/datasets/samsum) developed by Samsung, which consists of a collection of dialogues
along with brief summaries.

Let’s load it and look at an example:



In [ ]:
dataset_samsum = load_dataset("samsum")
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print(f"\nDialogue:")
print(dataset_samsum["test"][0]["dialogue"])
print(f"\nSummary:")
print(dataset_samsum["test"][0]["summary"])

The dialogues look like what you would expect from a chat via SMS or WhatsApp,
including emojis and placeholders for GIFs. The dialogue field contains the full text
and the summary the summarized dialogue.

Could a model that was fine-tuned on the
CNN/DailyMail dataset deal with that? 

Let’s find out!

##Evaluating PEGASUS on SAMSum

First we’ll run the same summarization pipeline with PEGASUS to see what the output
looks like.

In [ ]:
pipe_out = pipe(dataset_samsum["text"][0]["dialogue"])
print("Summary:")
print(pipe_out[0]["summary_text"].replace("  .<n>", ".\n"))

We can see that the model mostly tries to summarize by extracting the key sentences
from the dialogue. This probably worked relatively well on the CNN/DailyMail dataset,
but the summaries in SAMSum are more abstract.

Let’s confirm this by running
the full ROUGE evaluation on the test set:

In [ ]:
score = evaluate_summaries_pegasus(dataset_samsum["test"], 
                                   rouge_metric, 
                                   model, 
                                   tokenizer, 
                                   column_text="dialogue", 
                                   column_summary="summary", 
                                   batch_size=8)
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

pd.DataFrame.from_dict(rouge_dict, index=["pegasus"])

Well, the results aren’t great, but this is not unexpected since we’ve moved quite a bit
away from the CNN/DailyMail data distribution.

Nevertheless, setting up the evaluation
pipeline before training has two advantages: we can directly measure the success
of training with the metric and we have a good baseline.

Fine-tuning the model on
our dataset should result in an immediate improvement in the ROUGE metric, and if
that is not the case we’ll know something is wrong with our training loop.

##Fine-Tuning PEGASUS

Before we process the data for training, let’s have a quick look at the length distribution
of the input and outputs: